In [ ]:
import sys
sys.path.append("ANN_scripts")
import numpy as np
import mlflow as mlf
import mlflow.sklearn
import pandas as pd
import numpy as np
import torch
import torch.nn
from NeuralNetwork import NeuralNetwork
import os


### Definimos las variables de las métricas, los modelos, los parámetros y los sets de datos.

In [ ]:
variables = []
pkls = os.listdir("./pkl")
pkls.sort()
for pkl in pkls:
    variables.append(pd.read_pickle(f"./pkl/{pkl}"))

precision_test, recall_test, precision_train, recall_train, X_test_ann, X_test_xgb     = variables[:6] 
X_train_ann, X_train_xgb, accuracy_test_ann, accuracy_train_ann, ann_model, params_ann = variables[6:12] 
params_xgb, xgb_clf, y_test_ann, y_test_xgb, y_train_ann, y_train_xgb                  = variables[12:] 


### Se registran los modelos, las métricas y los parámetros.

In [ ]:
run_id = None

with mlf.start_run(run_name="Sleep Disorder Classifiers", description="ANN and XGB comparison") as run:
    mlf.log_metric("XGB_precision_train", precision_train*100)
    mlf.log_metric("XGB_recall_train", recall_train*100)
    mlf.log_metric("XGB_precision_test", precision_test*100)
    mlf.log_metric("XGB_recall_test", recall_test*100)
    mlf.log_metric("ANN_acuraccy_train", accuracy_train_ann)
    mlf.log_metric("ANN_acuraccy_test", accuracy_test_ann)
    mlf.log_params(params_xgb)
    mlf.log_params(params_ann)
    mlf.sklearn.log_model(xgb_clf, artifact_path="model_xgb")
    mlf.sklearn.log_model(ann_model, artifact_path="model_ann")

    run_id = run.info.run_id
    print("Model saved in run %s" % run_id)

### Cargamos el modelo XGB y realizamos una predicción

In [ ]:
model_uri = "runs:/{}/model_xgb".format(run_id)
loaded_xgb_model = mlflow.sklearn.load_model(model_uri)
print("---Predicciones XGB---\n",loaded_xgb_model.predict(X_test_xgb))

### Cargamos el modelo ANN y realizamos una predicción

In [ ]:
model_uri = "runs:/{}/model_ann".format(run_id)
loaded_ann_model = mlflow.sklearn.load_model(model_uri)
X_test_tensor  = torch.tensor(X_test_ann.values.astype(np.float32), dtype=torch.float32)
pred = loaded_ann_model(X_test_tensor)
predicted_labels = torch.argmax(pred, dim=1)
print("---Predicciones ANN---\n",predicted_labels)